### 介绍
```
    对空间音效做一个简要介绍，如果我们想要听到3D空间音效，有一个方法就是用很多扬声器布局构造一个顶级的播放环境，在合适的位置能够享受到极致的听觉体验；但是，无论传来的声音多么立体，声音只会从两个耳朵接受，所以我们能否实现在耳朵处，直接播放3D空间音效的声波到达的音频，直接实现3D音效；方法就是，相同的音频，到达左右耳，会在不同频率有所改变，也就是经过了不同冲激响应函数系统，这点微小的改变会使得耳朵分辨出方位；于是人们测出了不同位置到达耳朵时的冲击响应函数，做到空间音效处理。
```
    

### 一.前置准备
1. 首先安装一下可能会用到的库并导入
2. 完成基本的读入音频和写入音频

In [ ]:
!pip install numpy scipy matplotlib

In [2]:
import numpy as np
import wave
import scipy.io.wavfile as wav
from scipy.signal import resample, convolve


In [3]:

# 读取wav文件
def read_wav(filename):
    Fs, data = wav.read(filename)
    return Fs, data


In [4]:

# 写入wav文件
def write_wav(filename, Fs, data):
    wav.write(filename, Fs, data.astype(np.int16))


3. 重采样函数，用于改变音频的采样率

In [5]:

# 重采样函数
def resample_audio(data, original_fs, new_fs):
    num_samples = int(len(data) * float(new_fs) / original_fs)
    return resample(data, num_samples)


## 二.实现音频处理函数

1. 这里音频函数会接受两个声道的数据，两个声道的HRIR和前后采样率
2. 先进行重采样
3. 将左右声道和冲激串做一维卷积
4. 合成左右声道

In [6]:

# 处理单声道音频进行卷积
def process_audio(L,R, hrir_l, hrir_r, Fs, new_fs):
    # 重采样音频
    resampled_audio_L = resample_audio(L,Fs,new_fs)
    resampled_audio_R = resample_audio(R,Fs,new_fs)
    # 分别与左右HRIR卷积
    out_l = convolve(resampled_audio_L, hrir_l)
    out_r = convolve(resampled_audio_R, hrir_r)
    
    # 合成立体声
    out_stereo = np.column_stack((out_l, out_r))
    return out_stereo


### 三.进行实验测试
1. 读取HRIR数据用于对音频卷积处理
2. 对冲激串进行放缩操作，可以理解单位化，避免得到的音频数据过大

In [ ]:
L_path='full\\elev60\\L60e250a.wav'
R_path='full\\elev60\\R60e250a.wav'
# 读取HRIR数据
Fs_hrir, hrir_l = read_wav(L_path)
_, hrir_r = read_wav(R_path)
hrir_l,hrir_r=hrir_l*0.00005,hrir_r*0.00005
print(max(hrir_l),max(hrir_r))

3. 读取音频文件，得到左右声道

In [ ]:

# 读取音频文件
Fs, audio_data = read_wav('test.wav')
if len(audio_data[0])==2:
    audio_data_LL=[data[0] for data in audio_data]
    audio_data_RR=[data[1] for data in audio_data]
    print("双声道")
else:
    print('单声道')
print(max(audio_data_LL),max(audio_data_RR))

4. 实现音频处理

In [9]:
# 设置目标采样率
new_fs = 48000

# 处理音频，进行卷积并生成立体声
processed_audio = process_audio(audio_data_LL,audio_data_RR, hrir_l, hrir_r, Fs, new_fs)

# 将处理后的音频写入文件
write_wav('processed_audio.wav', new_fs, processed_audio)


In [ ]:
print(len(processed_audio))